<a href="https://colab.research.google.com/github/ctrivino1/YLearn/blob/main/YLearn_global_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/ctrivino1/YLearn.git

In [ ]:
# data
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
from ylearn.estimator_model.double_ml import DoubleML
from xgboost import XGBRegressor
import seaborn as sns
import matplotlib.pyplot as plt

housing = fetch_california_housing(as_frame=True)
data = housing.frame
outcome = housing.target_names[0]
data[outcome] = housing.target
data.head()

In [ ]:
from ylearn.estimator_model import double_ml
from ylearn import Why
# identify
why = Why()
r=why.identify(data, outcome,treatment_count_limit=2)

print('identified treatment:',r[0])
print('identified adjustment:',r[1])
print('identified covariate:',r[2])
print('identified instrument:',r[3])

In [ ]:
# Assuming 'your_outcome_column' is the outcome variable name
x_model = XGBRegressor()  # You can choose a different model if needed
y_model = XGBRegressor()  # You can choose a different model if needed

double_ml_model = double_ml.DoubleML(
    x_model=x_model,
    y_model=y_model,
    cf_fold=10,  # You can adjust the cross-fitting folds
    random_state=2022,
)

train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]
# Fit the DoubleML model (this is just an example, you need to specify the model and instruments)
double_ml_model.fit(data,outcome,treatment=r[0],covariate=r[2])

estimated_ate = double_ml_model.estimate(val_data,quantity='ATE')
print(f"Treatment{double_ml_model.treatment}")
print(f"Treatment ATE: {estimated_ate}")
y_intercept = double_ml_model.yx_model.intercept_
print(f"Y Intercept: {y_intercept}")
coefficents = double_ml_model.yx_model.coef_
print(f"coefficients: {coefficents}")

In [ ]:
double_ml.global_cov_before